# Graph

## 834. Sum of Distances in Tree

There is an undirected connected tree with n nodes labeled from 0 to n - 1 and n - 1 edges.

You are given the integer n and the array edges where edges[i] = [ai, bi] indicates that there is an edge between nodes ai and bi in the tree.

Return an array answer of length n where answer[i] is the sum of the distances between the ith node in the tree and all other nodes.

https://leetcode.com/problems/sum-of-distances-in-tree/description/

Example:
```
Input: n = 6, edges = [[0,1],[0,2],[2,3],[2,4],[2,5]]
Output: [8,12,6,10,10,10]
Explanation: 
The tree is shown above.
We can see that dist(0,1) + dist(0,2) + dist(0,3) + dist(0,4) + dist(0,5)
equals 1 + 1 + 2 + 2 + 2 = 8.
Hence, answer[0] = 8, and so on.

```


### 想法：DFS, Re-Root, DP

參考： https://github.com/wisdompeak/LeetCode/tree/master/Tree/834.Sum-of-Distances-in-Tree

這題是一個沒有cycle的連通圖
要知道任一個節點對應每個節點的距離和，可以使用「re-root」的概念<br>
主要就是要先知道根節點到每個節點的距離之和（可以深度走訪取得）<br>
然後根節點的相鄰子節點，到每個節點的距離和，就是包含自己與連接到的其他子樹節點的距離-1，與其他節點的距離+1,所以就是：<br>

非根節點對應每個節點的距離和：f(child) = f(parent) - (child子樹的節點數) + （N - child子樹的節點數）

所以還要知道每個節點的子樹節點的數量才行～（可以深度走訪取得）<br>


code執行方面：
- 為了DFS，建一個記錄走訪的的list，還有一個每個節點與相鄰節點的dictionary（{cur_node:[adjacent_nodes]}

- 為了儲存每個節點的子樹節點數量，建一個count的list，index是節點值，value是那個節點的子樹節點數量（包含自己）

- 建一個能計算子樹節點數量並填完count的dfs函數：遍歷當前節點的下一層節點，已經訪問過的就跳過，沒訪問過的，先標記在訪問清單中，然後遞迴訪問這個節點，看它有沒有下一層，已經到leaf node的話，就回傳1(子節點只算自己一個)，接著把當前節點以下的子樹節點數量通通加起來回傳，再把這個數放到count[i]之中。

- 建一個能計算根部節點到每個節點的路徑長度的總和的dfs函數：因為count有記錄到每個節點的子樹節點數量，把這個數扣1就是當前節點向下一層的路徑會被經過的次數（底下共幾個節點就會被加幾次），因此，把深度走訪過程之中的節點的「count[i]-1」全部相加起來的話，就是根節點到每一個節點的距離之總和了～～

- 依序呼叫上面建好的函數，以拿到count跟根節點到各節點的距離和，記得每次呼叫要初始化visited[]，然後走訪的第一個點要先標記為1

- 最後再建一個dfs函數，再次走訪，用動態規畫，把根節點算出的路徑總和，以「移根」的想法往子節點推，然後把每個節點的計算結果存到res[]中，這個res就是答案了

In [ ]:
from collections import defaultdict

class Solution:
    def sumOfDistancesInTree(self, n: int, edges: List[List[int]]) -> List[int]:
        # 移根的概念，先用DFS遍歷，得到root到每個節點的距離之和，以及每個節點對應的子數的節點個數
        # 非根節點對應每個節點的距離和：f(child) = f(parent) - (child子樹的節點數) + （N - child子樹的節點數）

        visited = [0 for _ in range(n)]
        count = [0 for _ in range(n)]

        # 建立每個節點與連結的節點的list
        next = defaultdict(list)
        for edge in edges:
            next[edge[0]].append(edge[1])
            next[edge[1]].append(edge[0])
        
        print(next)

        # 計算子樹節點數量(存在count中)
        def dfs(cur):
            ss = 1    # 尾端的最後子結果
            for x in next[cur]:
                if visited[x] != 0:
                    continue
                else:
                    visited[x] = 1
                    ss += dfs(x)
            count[cur] = ss
            return ss

        # 計算根節點到每個位置的距離和
        def dfs_path(cur):
            ss = 0 
            for x in next[cur]:
                if visited[x] != 0:
                    continue
                else:
                    visited[x] = 1
                    ss += dfs_path(x)
            ss += count[cur]-1   # 把每個節點往下的所有path加起來了
            return ss

        
        visited[0] = 1
        dfs(0)           #得到count

        visited = [0 for _ in range(n)]  # 訪問list歸零
        visited[0] = 1                   # 記得第一個點還是要標記已訪問！
        res = [0 for _ in range(n)]
        res[0] = dfs_path(0)   # 得到根節點到每個節點的路徑和
        print(res[0])
        print(count)

        def dfs_res(cur, total):
            for x in next[cur]:
                if visited[x] != 0:
                    continue
                else:
                    visited[x] = 1
                    a = count[x]
                    b = n - count[x]
                    res[x] = total + b - a
                    dfs_res(x, res[x])

        visited = [0 for _ in range(n)]  # 訪問list歸零
        visited[0] = 1
        dfs_res(0, res[0])

        return res